Author: Alkistis Pourtsidou, ICG Portsmouth

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this Jupyter notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

In [1]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np

#print('Using CAMB installed at '+ os.path.realpath(os.path.join(os.getcwd(),'..')))
#sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
import camb
from camb import model, initialpower

In [2]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [3]:
from scipy import integrate
from scipy import linalg

pi=np.pi

In [4]:
#Fiducial cosmological parameters
c=3e5
hubble=0.678
omegab=0.022*pow(hubble,-2)
omegac=0.119*pow(hubble,-2)
om0=omegac+omegab
H00=100*hubble
Ass=2.14e-9
nss = 0.968

gamma=0.545

print om0, H00

0.306732450988 67.8


In [5]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [6]:
#calculate results for these parameters
results = camb.get_results(pars)
#print pars

In [7]:
#Get matter power spectrum at z=0: P(k,z=0)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Non-Linear spectra (Halofit)
pars.NonLinear = model.NonLinear_both
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-6, maxkh=2.0, npoints = 200)

In [8]:
#Construct P(k,z=0) interpolating function, in units of Mpc (no h)
Pkz0 = interp1d(kh*hubble, pk[0]/pow(hubble,3))

In [9]:
#Redshift bins
zlist = np.arange(0.1,1.45,0.1)
ztest = zlist[2]
Nzbins = len(zlist)

print zlist
print "ztest =", ztest
print "Number of redshift bins =", Nzbins

[ 0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.   1.1  1.2  1.3  1.4]
ztest = 0.3
Number of redshift bins = 14


In [10]:
#Spatially flat Universe

#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
def Hz(zc):
    return Ez(zc)*H00
#Define the comoving distances
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)
def DA(zc):
    return rcom(zc)/(1+zc)

print ztest, rcom(ztest)
#print DA(ztest)

0.3 1232.20219137


In [11]:
#Define the growth function in LCDM
def fg(zz):
    omz=om0*pow(1+zz,3)/pow(Ez(zz),2)
    return pow(omz,gamma)

print fg(ztest)

0.68008016167


In [12]:
#Get the growth factor 
def Dg_dz(zz):
    return -fg(zz)/(1+zz)

def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(ans)

print Dgz(ztest)

0.853606528979


In [13]:
#fiducial bHI from Bull et al 2015
def bHI(zc):
    return 0.67+0.18*zc+0.05*pow(zc,2)

#fiducial OmHI from Crighton et al 2015
def OmHI(zc):
    return 0.0004*pow(1+zc,0.6)

print bHI(ztest), OmHI(ztest)

0.7285 0.000468194171289


In [14]:
#Construct P(k,μ,z) 
def beta(zc):
    return fg(zc)/bHI(zc)

def Pkz(kk,mu,zc):    
    return pow(1+beta(zc)*mu**2,2)*pow(Dgz(zc),2)*Pkz0(kk)

print Pkz(0.1,0.0,ztest)

8591.67784341


In [15]:
#mean brightness temperature [mK]
def Tb(zc):
    return 180.*OmHI(zc)*hubble*pow(1+zc,2)/Ez(zc)

print Tb(ztest)

0.0825858102674


In [16]:
#Construct P_HI(k,μ,z), Tb assumed known
def PHI(kk,mu,zc):
    return pow(bHI(zc),2)*Pkz(kk,mu,zc)

print PHI(0.1,0.0,ztest)

4559.70867955


In [17]:
#MeerKAT noise specs
Ndishes=64
Ddish=13.5*100 #cm
Nbeams=1

def thetab(zc):
    return 21*(1+zc)/Ddish

def omegapix(zc):
    return 1.13*pow(thetab(zc),2)

Area=5000.0 #deg^2
fsky=Area/44000
omegatot = Area*pow(pi/180,2)
ttotal = 3650*60*60 #5 months in sec

#Tsys per band [in mK] from the latest specs document
Tsyslist = [23.5*1e3,23.0*1e3,23.0*1e3,28.0*1e3,29.0*1e3,30.0*1e3,28.5*1e3,29.5*1e3,\
            31.0*1e3,33.0*1e3,34.0*1e3,35.0*1e3,37.0*1e3,38.0*1e3] 

def tobs(zc):
    return ttotal*(omegapix(zc)/omegatot)*Ndishes*Nbeams

Dzbin = 0.1 #redshift bin width
dfpix = 50*1e3 #frequency resolution in Hz
midfreq = 1420.4e6 #Hz

def dzpix(zc):
    return pow(1+zc,2)*dfpix/midfreq
def sigpix(zc,Tsys):
    return Tsys/np.sqrt(dfpix*tobs(zc)) 
def dVpixdz(zz):    
    return c*pow(rcom(zz),2)/(H00*Ez(zz))
def Vpix(zc):
    return omegapix(zc)*scipy.integrate.romberg(dVpixdz,zc-dzpix(zc)/2,zc+dzpix(zc)/2)

def Wsq(kk,zc):
    return np.exp(-pow(kk,2)*pow(rcom(zc),2)*pow(thetab(zc),2)/(8*np.log(2)))

#note division by Tb^2
def Pnoise(kk,zc,Tsys):
    return pow(sigpix(zc,Tsys),2)*Vpix(zc)*pow(Wsq(kk,zc),-1.)/pow(Tb(zc),2)

In [18]:
def kmin(zc):
    return 2*pi/np.sqrt(pow(rcom(zc),2)*omegatot)
#non-linear cutoff (Smith et al 2003), alternatively could use instrumental cutoff
def kmax(zc):
    return 0.14*pow(1+zc,2/(2+nss))

In [19]:
#survey (bin) volume [Mpc^3]
def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

#effective volume going in the Fisher matrix
def Veff(kk,mu,zc):
    return Vsur(zc)*(PHI(kk,mu,zc)/(PHI(kk,mu,zc)+Pnoise(kk,zc,Tsys)))**2

print "%.4g" % Vsur(ztest)

8.782e+08


In [20]:
#Fisher matrix parameters 
params = ["0:fsig8","1:bsig8","2:DA","3:H"]

In [21]:
#Fisher matrix derivatives

def dlnP_dlnfsig8(kk,mu,zc):
    return 2*mu**2*fg(zc)/(bHI(zc)+mu**2*fg(zc))

def dlnP_dlnbsig8(kk,mu,zc):
    return 2*bHI(zc)/(bHI(zc)+mu**2*fg(zc))

def dlnP_dlnDA(kk,mu,zc):
    dk = (kmax(zc)-kmin(zc))/400
    return (-2.0+4*mu**2*(1-mu**2)*fg(zc)/(bHI(zc)+mu**2*fg(zc))
            -kk*(1-mu**2)*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk))

def dlnP_dlnH(kk,mu,zc):
    dk = (kmax(zc)-kmin(zc))/400
    return (1.0+4*mu**2*(1-mu**2)*fg(zc)/(bHI(zc)+mu**2*fg(zc))
            +kk*mu**2*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk))

In [22]:
def dF(kk,mu):
    return (1./(8*pi*pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)    

In [23]:
#2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.simps(dfun.T[i], mugrid) for i in range(kgrid.size)]
    return scipy.integrate.simps(muint, kgrid)

In [24]:
mugrid = np.linspace(-1., 1., 200) 

In [25]:
#%%time
#   Fisher matrix   # 

Npar = 4
#create array of zeros
s = (Npar,Npar)

for zi in range(0,Nzbins):
    zc = zlist[zi]
    Tsys = Tsyslist[zi]
    kgrid = np.linspace(kmin(zc), kmax(zc), 400)
    K, MU = np.meshgrid(kgrid, mugrid)
    Fishermat = np.zeros(s)
    for i in range(0,Npar):  
        def deriv_i(kk,mu,zc):
            if i==0:  return dlnP_dlnfsig8(kk,mu,zc)
            elif i==1:  return dlnP_dlnbsig8(kk,mu,zc)
            elif i==2:  return dlnP_dlnDA(kk,mu,zc)
            elif i==3:  return dlnP_dlnH(kk,mu,zc)
            else: print "out of range"
        for  j in range(0,Npar):
            if j>=i:
                def deriv_j(kk,mu,zc):
                    if j==0:  return dlnP_dlnfsig8(kk,mu,zc)
                    elif j==1:  return dlnP_dlnbsig8(kk,mu,zc)
                    elif j==2:  return dlnP_dlnDA(kk,mu,zc)
                    elif j==3:  return dlnP_dlnH(kk,mu,zc)
                    else: print "index out of range" 
                Fishermat[i][j] = integrate2D(dF(K,MU),kgrid,mugrid)
            else: Fishermat[i,j] = Fishermat[j,i]
    #if zi==4:
    #    for i in range(0,Npar):
    #        for  j in range(0,Npar):
    #            print i,j,linalg.inv(Fishermat)[i,j]
            
    #print z_bin and fractional error on fsig8,bsig8,DA,H       
    print zc, np.sqrt(linalg.inv(Fishermat)[0,0]),\
              np.sqrt(linalg.inv(Fishermat)[1,1]),\
              np.sqrt(linalg.inv(Fishermat)[2,2]),\
              np.sqrt(linalg.inv(Fishermat)[3,3])

0.1 0.0757255031546 0.048656815648 0.0532370731183 0.0644301058304
0.2 0.0415451860254 0.0276800940362 0.0304122348735 0.0354614266829
0.3 0.040523115381 0.0269770804728 0.0293600120647 0.0330691889682
0.4 0.0552714556581 0.0379820741228 0.0404549817158 0.0445025387887
0.5 0.0671288084322 0.0491359315357 0.0504762834103 0.0556969080034
0.6 0.0823763170213 0.0651440379076 0.0657160482729 0.0689610094427
0.7 0.103927783012 0.0892016851666 0.0870101999881 0.0893573202746
0.8 0.139626896886 0.126175288463 0.115980170279 0.124029614779
0.9 0.161129979866 0.146305981659 0.13011651934 0.145097051832
1.0 0.177851042477 0.160028328941 0.140705278981 0.158561919685
1.1 0.196597628282 0.175505923048 0.153365197523 0.171840913789
1.2 0.222892254941 0.19855714825 0.172268191654 0.190587666038
1.3 0.264177729207 0.236461209306 0.203031653853 0.220667734733
1.4 0.311011695543 0.281095046432 0.237914390802 0.255310417693
